In [1]:
from qiskit import Aer, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.aqua.components.optimizers import COBYLA
from qiskit import Aer, execute
import numpy as np
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000
np.random.seed(199999)

In [2]:
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

In [3]:
target_distr

array([0.55730187, 0.44269813])

In [4]:

def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [5]:
def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost


In [6]:
# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=100000, tol=0.00001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.55730187 0.44269813]
Obtained Distribution: [0.5483, 0.4517]
Output Error (Manhattan Distance): 0.022803731882912293
Parameters Found: [1.47409606 1.80183461 0.36315499]


In [9]:
from qiskit.aqua.components.variational_forms import RYRZ
entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = RYRZ(num_qubits=2, depth=1, entanglement=entanglement)
    if entanglement == "linear":
        print("=============Linear Entanglement:=============")
    else:
        print("=============Full Entanglement:=============")
    # We initialize all parameters to 0 for this demonstration
    print(form.construct_circuit([0] * form.num_parameters).draw(line_length=100))
    print()

=============Linear Entanglement:=============
        ┌───────────┐┌───────┐ ░                               ░ ┌───────────┐┌───────┐ ░ 
q_0: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░───────────────■───────────────░─┤ U3(0,0,0) ├┤ U1(0) ├─░─
        ├───────────┤├───────┤ ░ ┌──────────┐┌─┴─┐┌──────────┐ ░ ├───────────┤├───────┤ ░ 
q_1: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░─┤ U2(0,pi) ├┤ X ├┤ U2(0,pi) ├─░─┤ U3(0,0,0) ├┤ U1(0) ├─░─
        └───────────┘└───────┘ ░ └──────────┘└───┘└──────────┘ ░ └───────────┘└───────┘ ░ 

=============Full Entanglement:=============
        ┌───────────┐┌───────┐ ░                               ░ ┌───────────┐┌───────┐ ░ 
q_0: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░───────────────■───────────────░─┤ U3(0,0,0) ├┤ U1(0) ├─░─
        ├───────────┤├───────┤ ░ ┌──────────┐┌─┴─┐┌──────────┐ ░ ├───────────┤├───────┤ ░ 
q_1: |0>┤ U3(0,0,0) ├┤ U1(0) ├─░─┤ U2(0,pi) ├┤ X ├┤ U2(0,pi) ├─░─┤ U3(0,0,0) ├┤ U1(0) ├─░─
        └───────────┘└───────┘ ░ └──────────┘└───┘└──────────┘ ░ └───────────┘└───────┘ 

/Users/brett/anaconda3/lib/python3.7/site-packages/qiskit/circuit/quantumcircuit.py:861: DeprecationWarning: The parameter "line_length" is being replaced by "fold"
  ax=ax)
/Users/brett/anaconda3/lib/python3.7/site-packages/qiskit/circuit/quantumcircuit.py:861: DeprecationWarning: The parameter "line_length" is being replaced by "fold"
  ax=ax)
